# Lorenz 96 Ginelli Workbook

This notebook is a space for quick investigations & current implementation notes. Less formal.

In [1]:
# Imports

import numpy as np
import time as tm
import pickle
import xarray as xr
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

In [3]:
# Progress Bar Import

def in_notebook():
    """
    Returns ``True`` if the module is running in IPython kernel,
    ``False`` if in IPython shell or other Python shell.
    """
    return 'ipykernel' in sys.modules

if in_notebook():
    from tqdm.notebook import tqdm #tqdm is progress bar
else:
    from tqdm import tqdm

### Notes


## $k_a$ Timings

Timing the effect of $k_a$.

In [2]:
ka_timings = [pickle.load(open(f'ka-timings/{i}/timings.p', 'rb')) for i in [1, 2, 3]]
parameters = []

for i in [1, 2, 3]:
    ds = xr.open_dataset(f'ka-timings/{i}/step5/LyapObs1.nc')
    print(ds.attrs)
    parameters.append(ds.attrs)
    ds.close()

{'h': 1, 'Fs': 10, 'Ff': 6, 'c': 10, 'J': 10, 'K': 36, 'Number of variables': 396, 'b': 10.0, 'dt': 0.001, 'tau': 0.01, 'transient': 1.0, 'ka': 250, 'kb': 250, 'kc': 250}
{'h': 1, 'Fs': 10, 'Ff': 6, 'c': 10, 'J': 10, 'K': 36, 'Number of variables': 396, 'b': 10.0, 'dt': 0.001, 'tau': 0.01, 'transient': 1.0, 'ka': 500, 'kb': 250, 'kc': 250}
{'h': 1, 'Fs': 10, 'Ff': 6, 'c': 10, 'J': 10, 'K': 36, 'Number of variables': 396, 'b': 10.0, 'dt': 0.001, 'tau': 0.01, 'transient': 1.0, 'ka': 1000, 'kb': 250, 'kc': 250}


In [15]:
data_list = [xr.open_dataset(f'ka-timings/{i}/step5/LyapObs1.nc') for i in [1, 2, 3]]

In [21]:
def spectra_parameter(data_list, parameter_key, geometry='C', save = 'None'):
    """Plots Lyapunov spectra for a varying parameter.
    data_list, list of data arrays containing lyapunov observations
    parameter_key, the parameter from data.attrs that we're investigating.
    geometry, CLEs or BLEs.
    save, file save name
    """

    for data in data_list:
        
        attr = data.attrs[parameter_key]
       
        if (geometry == 'C'):
            spectra = data.ftcle.mean(dim='time', skipna=True)
            label = f'{parameter_key}$={attr:.2f}$'
        else: # then FTBLE
            spectra = data.ftble.mean(dim='time', skipna=True)
            label = f'{parameter_key}$={attr:.2f}$'


        plt.plot(spectra, label=label)

    plt.legend()
    plt.title(f'FT{geometry}LE Means')
    plt.xlabel('LE Index') 
    
    if (save == 'None'):
        plt.show()
    else:
        print('Saving')
        plt.savefig(save, dpi=1200)
        plt.show()
    